In [1]:
import pandas as pd
from pandas import DataFrame
import os
import sys
sys.path.append('/home/bouyang/Software/Nanokit/Python/PythonScript')
from SISSOAna import __readdata, __getFeatureMat, rankedTransform
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

prefix = os.getcwd()

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
with open('KeyFeatureNames.txt','r') as fid:
    key_feature_names = eval(fid.read())

In [3]:
train_df.head()

,CompName,NewEhullLst,EWaldSumLst,EFormLst,RDiffLst,RDiffStdLst,RDiffNaLst,EleNegStdLst,XDiffLst,AnionXStdLst,XWithNaLst,ChgDiffLst,ChgStdLst,AnionChgStdLst,NNaLst,RRatioNaLst
0,MgZr(SO4)3,-60.260567,-65.921154,-1.891755,0.000,0.000000,-0.300000,0.010000,0.02,0.0,0.390000,2.0,1.000000,0.0,0.0,0.705882
1,MgTi(SO4)3,-27.986572,-66.524285,-1.828661,0.115,0.057500,-0.357500,0.115000,0.23,0.0,0.495000,2.0,1.000000,0.0,0.0,0.649510
2,MgSn(SO4)3,-30.640966,-66.032051,-1.554568,0.030,0.015000,-0.315000,0.325000,0.65,0.0,0.705000,2.0,1.000000,0.0,0.0,0.691176
3,Mg4Nb2(SO4)9,-42.783127,-66.565193,-1.750108,0.080,0.037712,-0.326667,0.136707,0.29,0.0,0.476667,3.0,1.414214,0.0,0.0,0.679739
4,ZrZn(SO4)3,-40.419096,-65.810136,-1.699355,0.020,0.010000,-0.290000,0.160000,0.32,0.0,0.560000,2.0,1.000000,0.0,0.0,0.715686


In [4]:
test_df.head()

,CompName,NewEhullLst,EWaldSumLst,EFormLst,RDiffLst,RDiffStdLst,RDiffNaLst,EleNegStdLst,XDiffLst,AnionXStdLst,XWithNaLst,ChgDiffLst,ChgStdLst,AnionChgStdLst,NNaLst,RRatioNaLst
0,MgGe(SO4)3,-9.215538,-66.427225,-1.520901,0.19,0.095000,-0.395000,0.350000,0.70,0.0,0.730000,2.0,1.000000,0.0,0.0,0.612745
1,Mg4Ta2(SO4)9,-48.215659,-66.611213,-1.785828,0.08,0.037712,-0.326667,0.089567,0.19,0.0,0.443333,3.0,1.414214,0.0,0.0,0.679739
2,ZnSn(SO4)3,-21.229072,-65.840156,-1.350749,0.05,0.025000,-0.305000,0.155000,0.31,0.0,0.875000,2.0,1.000000,0.0,0.0,0.700980
3,ZnGe(SO4)3,-25.113430,-66.215531,-1.285539,0.21,0.105000,-0.385000,0.180000,0.36,0.0,0.900000,2.0,1.000000,0.0,0.0,0.622549
4,CaHf(SO4)3,-105.503851,-65.771901,-1.947371,0.29,0.145000,-0.165000,0.150000,0.30,0.0,0.220000,2.0,1.000000,0.0,0.0,0.838235


In [5]:
print(key_feature_names,len(key_feature_names))

['NewEhullLst', 'EWaldSumLst', 'EFormLst', 'RDiffLst', 'RDiffStdLst', 'RDiffNaLst', 'EleNegStdLst', 'XDiffLst', 'AnionXStdLst', 'XWithNaLst', 'ChgDiffLst', 'ChgStdLst', 'AnionChgStdLst', 'NNaLst', 'RRatioNaLst'] 15


In [6]:
def apply_rank_model(X_train,Y_train,X_test,Y_test,BaseNames):
    """
    Apply the ranking SVM model and get merits we need for analysis
    """
    feature_comp = ['(cbrt(NNaLst)+(AnionChgStdLst)^2)', '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))']
    
    RankedFeatTrain,RankedTargetTrain,RankedFeatTest,RankedTargetTest = X_train,Y_train,X_test,Y_test
    TrainFeatureMat=__getFeatureMat(X_train,BaseNames,feature_comp)
    TestFeatureMat=__getFeatureMat(X_test,BaseNames,feature_comp)
    print(TrainFeatureMat.shape, TestFeatureMat.shape)
    TransTrainFeatMat,TransTrainTargets=rankedTransform(TrainFeatureMat,RankedTargetTrain);
    TransTestFeatMat,TransTestTargets=rankedTransform(TestFeatureMat,RankedTargetTest);
    Clf=LinearSVC(penalty='l2',loss='squared_hinge',dual=False,tol=0.0001,C=1.0,multi_class='ovr',\
              fit_intercept=True,intercept_scaling=1,class_weight=None,verbose=0,\
              random_state=None,max_iter=1000)
    
    Clf.fit(TransTrainFeatMat,TransTrainTargets);
    Y_test_predict=Clf.predict(TransTestFeatMat);
    Y_train_predict=Clf.predict(TransTrainFeatMat);
    TrainScore=accuracy_score(TransTrainTargets,Y_train_predict);
    Score=accuracy_score(TransTestTargets,Y_test_predict);
    F1Train=f1_score(TransTrainTargets,Y_train_predict);
    F1Test=f1_score(TransTestTargets,Y_test_predict);
    F1TrainVals=f1_score(TransTrainTargets,Y_train_predict,average=None);
    F1TestVals=f1_score(TransTestTargets,Y_test_predict,average=None);
    
    print('Feature comp index has CV score {}'.format(Score));
    print('Feature comp index has train score {}'.format(TrainScore));
    print('Feature comp index has CV F1 {}'.format(F1Test));
    print('Feature comp index has train F1 {}'.format(F1Train));
    
    return Score, F1Test

In [7]:
feature_comp = ['(cbrt(NNaLst)+(AnionChgStdLst)^2)', '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))']

In [8]:
X_train, X_test, Y_train, Y_test = np.float32(train_df.to_numpy()[:,2:]), np.float32(test_df.to_numpy()[:,2:]), \
                                   np.float32(train_df.to_numpy()[:,1]), np.float32(test_df.to_numpy()[:,1])

In [9]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(3104, 14) (777, 14) (3104,) (777,)


In [10]:
print(X_train[:3,:])

[[-6.5921150e+01 -1.8917547e+00  0.0000000e+00  0.0000000e+00
  -3.0000001e-01  9.9999998e-03  2.0000000e-02  0.0000000e+00
   3.8999999e-01  2.0000000e+00  1.0000000e+00  0.0000000e+00
   0.0000000e+00  7.0588237e-01]
 [-6.6524284e+01 -1.8286612e+00  1.1500000e-01  5.7500001e-02
  -3.5749999e-01  1.1500000e-01  2.3000000e-01  0.0000000e+00
   4.9500000e-01  2.0000000e+00  1.0000000e+00  0.0000000e+00
   0.0000000e+00  6.4950979e-01]
 [-6.6032051e+01 -1.5545683e+00  2.9999999e-02  1.5000000e-02
  -3.1500000e-01  3.2499999e-01  6.4999998e-01  0.0000000e+00
   7.0499998e-01  2.0000000e+00  1.0000000e+00  0.0000000e+00
   0.0000000e+00  6.9117647e-01]]


In [11]:
TrainFeatureMat=__getFeatureMat(X_train,key_feature_names[1:],feature_comp)
TestFeatureMat=__getFeatureMat(X_test,key_feature_names[1:],feature_comp)

In [12]:
RankedFeatTrain,RankedTargetTrain,RankedFeatTest,RankedTargetTest = X_train,Y_train,X_test,Y_test 
TransTrainFeatMat,TransTrainTargets=rankedTransform(TrainFeatureMat,RankedTargetTrain);
TransTestFeatMat,TransTestTargets=rankedTransform(TestFeatureMat,RankedTargetTest);

NewFeatMat has the shape (4815856, 2)
NewTarget has the shape (4815856,)
NewFeatMat has the shape (301476, 2)
NewTarget has the shape (301476,)


In [13]:
Clf=LinearSVC(penalty='l2',loss='squared_hinge',dual=False,tol=0.0001,C=1.0,multi_class='ovr',\
              fit_intercept=True,intercept_scaling=1,class_weight=None,verbose=0,\
              random_state=None,max_iter=1000)
Clf.fit(TransTrainFeatMat,TransTrainTargets);
Y_test_predict=Clf.predict(TransTestFeatMat);
Y_train_predict=Clf.predict(TransTrainFeatMat);
TrainScore=accuracy_score(TransTrainTargets,Y_train_predict);
Score=accuracy_score(TransTestTargets,Y_test_predict);
F1Train=f1_score(TransTrainTargets,Y_train_predict);
F1Test=f1_score(TransTestTargets,Y_test_predict);
F1TrainVals=f1_score(TransTrainTargets,Y_train_predict,average=None);
F1TestVals=f1_score(TransTestTargets,Y_test_predict,average=None);

print('Feature comp index has CV score {}'.format(Score));
print('Feature comp index has train score {}'.format(TrainScore));
print('Feature comp index has CV F1 {}'.format(F1Test));
print('Feature comp index has train F1 {}'.format(F1Train));

Feature comp index has CV score 0.8470923058551925
Feature comp index has train score 0.8416887880368517
Feature comp index has CV F1 0.7232116052021662
Feature comp index has train F1 0.7201762029167


In [14]:
print(TrainFeatureMat.shape)

(3104, 2)


In [15]:
print(TrainFeatureMat[2:4,:])

[[ 0.         46.10945129]
 [ 0.         79.65130615]]


In [16]:
print(train_df['CompName'].to_list())

['MgZr(SO4)3', 'MgTi(SO4)3', 'MgSn(SO4)3', 'Mg4Nb2(SO4)9', 'ZrZn(SO4)3', 'TiZn(SO4)3', 'Nb2Zn4(SO4)9', 'Ta2Zn4(SO4)9', 'CaZr(SO4)3', 'CaTi(SO4)3', 'CaSn(SO4)3', 'Ca4Nb2(SO4)9', 'Ca4Ta2(SO4)9', 'LaGa(SO4)3', 'YSc(SO4)3', 'ScIn(SO4)3', 'Y2(SO4)3', 'MgZr3P2(SO6)4', 'MgZr3SiS5O24', 'MgTi3P2(SO6)4', 'MgSn3P2(SO6)4', 'Hf3MgP2(SO6)4', 'Mg2Nb2SiS5O24', 'Zr3ZnP2(SO6)4', 'Ti3ZnP2(SO6)4', 'NbZnP(SO6)2', 'Nb2Zn2SiS5O24', 'TaZnP(SO6)2', 'Ta2Zn2SiS5O24', 'CaZr3P2(SO6)4', 'CaZr3SiS5O24', 'CaTi3P2(SO6)4', 'CaTi3SiS5O24', 'CaSn3P2(SO6)4', 'CaHf3P2(SO6)4', 'GaSnP(SO6)2', 'HfGaP(SO6)2', 'Hf2Ga2SiS5O24', 'NbGa3P2(SO6)4', 'TaGa3P2(SO6)4', 'HfAlP(SO6)2', 'Hf2Al2SiS5O24', 'TaAl3P2(SO6)4', 'ZrScP(SO6)2', 'Zr2Sc2SiS5O24', 'ScTiP(SO6)2', 'HfScP(SO6)2', 'Hf2Sc2SiS5O24', 'Sc3NbP2(SO6)4', 'Sc3TaP2(SO6)4', 'Y2Zr2SiS5O24', 'Y2Hf2SiS5O24', 'Zr2In2SiS5O24', 'HfInP(SO6)2', 'NbIn3P2(SO6)4', 'LaZrP(SO6)2', 'LaTiP(SO6)2', 'La2Ti2SiS5O24', 'LaSnP(SO6)2', 'La2SiSn2S5O24', 'La2SiGe2S5O24', 'Mg2Nb4P6(SO12)3', 'Mg2Ta4P6(SO12)3

In [17]:
train_2D_df = DataFrame.from_dict({
    'CompName':train_df['CompName'].to_numpy(),
    'Ehull':RankedTargetTrain,
    '(cbrt(NNaLst)+(AnionChgStdLst)^2)':TrainFeatureMat[:,0],
    '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))':TrainFeatureMat[:,1]
})

test_2D_df = DataFrame.from_dict({
    'CompName':test_df['CompName'].to_numpy(),
    'Ehull':RankedTargetTest,
    '(cbrt(NNaLst)+(AnionChgStdLst)^2)':TestFeatureMat[:,0],
    '((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))':TestFeatureMat[:,1]
})

In [18]:
train_2D_df.head()

,CompName,Ehull,(cbrt(NNaLst)+(AnionChgStdLst)^2),((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))
0,MgZr(SO4)3,-60.260567,0.0,0.000000
1,MgTi(SO4)3,-27.986572,0.0,125.960243
2,MgSn(SO4)3,-30.640966,0.0,46.109451
3,Mg4Nb2(SO4)9,-42.783127,0.0,79.651306
4,ZrZn(SO4)3,-40.419098,0.0,24.253452


In [19]:
test_2D_df.head()

,CompName,Ehull,(cbrt(NNaLst)+(AnionChgStdLst)^2),((EWaldSumLst)^2*(XWithNaLst*RDiffStdLst))
0,MgGe(SO4)3,-9.215537,0.0,306.012177
1,Mg4Ta2(SO4)9,-48.215660,0.0,74.183754
2,ZnSn(SO4)3,-21.229073,0.0,94.826508
3,ZnGe(SO4)3,-25.113430,0.0,414.334930
4,CaHf(SO4)3,-105.503853,0.0,137.997589


In [20]:
train_2D_df.to_csv('train_2D.csv',index=False)
test_2D_df.to_csv('test_2D.csv',index=False)